# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
data_df = pd.read_csv('../WeatherPy/output_data/cities.csv')
locations = data_df[['Latitude','Longitude']]
hum = data_df['Humidity'].astype(float)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=hum, dissipating=False,
                                  max_intensity=100, point_radius=3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
narrow_df = data_df[data_df['Max Temp']<80] 
narrow_df = narrow_df[data_df['Max Temp']>70]  
narrow_df = narrow_df[data_df['Wind Speed']<10] 
narrow_df = narrow_df[data_df['Cloudiness']==0]
narrow_df

C:\Users\argen\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
C:\Users\argen\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\argen\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


,City,Longitude,Latitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,busselton,115.3333,-33.6500,73.31,42,0,1.95,AU,1613100798
68,whitianga,175.7000,-36.8333,74.82,42,0,6.60,NZ,1613100906
103,viamao,-51.0233,-30.0811,75.20,88,0,8.05,BR,1613100815
111,atuona,-139.0333,-9.8000,79.66,74,0,9.91,PF,1613100914
114,mandurah,115.7000,-32.5500,77.00,49,0,1.99,AU,1613100915
212,arica,-70.3042,-18.4750,71.60,64,0,3.44,CL,1613100846
229,mecca,39.8261,21.4267,70.93,65,0,0.02,SA,1613100830
318,tahoua,5.2692,14.8888,71.60,13,0,4.61,NE,1613100942
511,pochutla,-96.4661,15.7432,77.00,82,0,1.99,MX,1613100968
520,det udom,105.0784,14.9060,73.99,60,0,6.91,TH,1613100969


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = narrow_df
hotel_df['Hotel Name']=0
hotel_df

,City,Longitude,Latitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
1,busselton,115.3333,-33.6500,73.31,42,0,1.95,AU,1613100798,0
68,whitianga,175.7000,-36.8333,74.82,42,0,6.60,NZ,1613100906,0
103,viamao,-51.0233,-30.0811,75.20,88,0,8.05,BR,1613100815,0
111,atuona,-139.0333,-9.8000,79.66,74,0,9.91,PF,1613100914,0
114,mandurah,115.7000,-32.5500,77.00,49,0,1.99,AU,1613100915,0
212,arica,-70.3042,-18.4750,71.60,64,0,3.44,CL,1613100846,0
229,mecca,39.8261,21.4267,70.93,65,0,0.02,SA,1613100830,0
318,tahoua,5.2692,14.8888,71.60,13,0,4.61,NE,1613100942,0
511,pochutla,-96.4661,15.7432,77.00,82,0,1.99,MX,1613100968,0
520,det udom,105.0784,14.9060,73.99,60,0,6.91,TH,1613100969,0


In [6]:
locations = hotel_df[["Latitude", "Longitude"]]

target_radius = 5000
target_type = "lodging"

hotel_names=[]
city_name=[]
country_name=[]
lat=[]
lon=[]

for index, row in hotel_df.iterrows():
    target_coordinates = str(row['Latitude']) + ',' + str(row['Longitude'])

    params = {
    "location": target_coordinates,
    "radius": target_radius,
    "types": target_type,
    "key": g_key}

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    response = requests.get(base_url, params=params)

    places_data = response.json()
    
    try:
        hotel_name = places_data['results'][0]['name']
    
        hotel_names.append(hotel_name)
        city_name.append(row['City'])
        country_name.append(row['Country'])
        lat.append(row['Latitude'])
        lon.append(row['Longitude'])
    except:
        continue

In [7]:
hotel_df2 = pd.DataFrame({'Hotel Name':hotel_names,
                          'City':city_name,
                          'Country': country_name,
                          'Latitude':lat,
                          'Longitude':lon})
hotel_df2

,Hotel Name,City,Country,Latitude,Longitude


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df2.iterrows()]
locations = hotel_df2[["Latitude", "Longitude"]]

In [9]:
# Add marker layer ontop of heat map
marker_locations = locations

fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations,info_box_content=hotel_info)
fig.add_layer(markers)
fig

# Display figure

Figure(layout=FigureLayout(height='420px'))